In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [5]:
import PPIGraph

# Read per cell data

In [11]:
# Read PPI
PPI_save_path =  data_dir / '13cyc' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,24782.000000,24782.000000,24782.000000,24782.000000
mean,451.891453,8.109111,2.371883,5.737229
std,264.699437,10.532796,4.842329,9.876917
min,21.000000,1.000000,0.000000,-157.000000
25%,227.000000,2.000000,0.000000,1.000000
50%,431.500000,5.000000,1.000000,3.000000
75%,649.000000,9.000000,3.000000,7.000000
max,1071.000000,232.000000,205.000000,232.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_19408\3196296033.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,2236.000000,2236.000000,2236.000000
mean,89.874776,26.288014,63.586762
std,49.101315,27.687732,53.779516
min,1.000000,0.000000,-434.000000
25%,59.000000,0.000000,29.000000
50%,81.000000,25.000000,56.000000
75%,109.000000,41.000000,89.000000
max,748.000000,577.000000,748.000000


In [16]:
# Filter out by maximum number of counts per cell
min_count = 0
max_count = 50

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
# df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cyto'] >= 0).all())

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,21982.000000,21982.000000,21982.000000,21982.000000
mean,451.329133,7.261441,2.386680,4.874761
std,263.249013,8.066408,4.827968,7.495634
min,21.000000,1.000000,0.000000,-157.000000
25%,231.000000,2.000000,0.000000,1.000000
50%,430.000000,4.000000,1.000000,3.000000
75%,649.000000,9.000000,3.000000,7.000000
max,1071.000000,49.000000,205.000000,49.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_19408\2030770136.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,2007.000000,2007.000000,2007.000000
mean,79.532138,26.140508,53.391629
std,32.651668,27.234153,40.010580
min,1.000000,0.000000,-434.000000
25%,57.000000,0.000000,27.000000
50%,77.000000,25.000000,51.000000
75%,99.000000,41.000000,79.000000
max,208.000000,577.000000,194.000000


# ML model

In [17]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [18]:
condition = 'ML'
project_name = f'PLA_13PPI_121923_{condition}'

In [19]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [20]:
df_count

Cell                                                   \
PPI               AKT/Mtor Bim/Tom20 Cyclin D1/CDK4 Cyclin E/CDK2 EGFR/GRB2   
Condition FOV Id                                                              
100nM     FW1 36       0.0       4.0            8.0           1.0       8.0   
              37       5.0       2.0            8.0           3.0      10.0   
              41       3.0       3.0            7.0           5.0       5.0   
              47       2.0       1.0            4.0           4.0       8.0   
              53       3.0       3.0            4.0           6.0      11.0   
...                    ...       ...            ...           ...       ...   
control   FW2 599      1.0       5.0            7.0           6.0      25.0   
              602      6.0       7.0           17.0           5.0      17.0   
              605      2.0       1.0            6.0           1.0       7.0   
              608      4.0       1.0            7.0          11.0      12.0   
              609      3.0       0.0           12.0           6.0      10.0   

                                                                          ...  \
PPI               FoxO1/AKT Mcl-1/BAK NF-Kb/p-P90rsk SIRT1/P53 Sox2/Oct4  ...   
Condition FOV Id                                                          ...   
100nM     FW1 36        8.0       4.0            5.0       0.0       1.0  ...   
              37        3.0       2.0            4.0       0.0       3.0  ...   
              41        3.0       5.0            2.0       0.0       0.0  ...   
              47        4.0       4.0            2.0       0.0       1.0  ...   
              53        8.0       2.0            5.0       0.0       6.0  ...   
...                     ...       ...            ...       ...       ...  ...   
control   FW2 599       2.0       3.0            5.0       0.0      12.0  ...   
              602      15.0       7.0            9.0       1.0       6.0  ...   
              605       4.0       1.0            0.0       0.0       3.0  ...   
              608       4.0       4.0            9.0       0.0       4.0  ...   
              609       7.0       5.0            3.0       0.0       3.0  ...   

                           Cyto                                               \
PPI               Cyclin E/CDK2 EGFR/GRB2 FoxO1/AKT Mcl-1/BAK NF-Kb/p-P90rsk   
Condition FOV Id                                                               
100nM     FW1 36            0.0       6.0       6.0       4.0            5.0   
              37            1.0       6.0      -3.0       1.0            1.0   
              41            5.0       3.0       3.0       4.0            0.0   
              47            1.0       4.0      -2.0       1.0           -8.0   
              53            1.0       8.0       0.0      -1.0           -3.0   
...                         ...       ...       ...       ...            ...   
control   FW2 599           6.0      25.0       2.0       3.0            5.0   
              602           5.0      17.0      15.0       7.0            9.0   
              605           1.0       7.0       4.0       1.0            0.0   
              608          11.0      12.0       4.0       4.0            9.0   
              609           6.0      10.0       7.0       5.0            3.0   

                                                                       
PPI               SIRT1/P53 Sox2/Oct4 TEAD1/YAP TRAIL/DR5 p-ERK/c-MYC  
Condition FOV Id                                                       
100nM     FW1 36        0.0       0.0       1.0       7.0        11.0  
              37        0.0       2.0       0.0       0.0         8.0  
              41        0.0       0.0       0.0       3.0        40.0  
              47        0.0      -2.0       0.0      -4.0         4.0  
              53        0.0       6.0       1.0      -3.0        13.0  
...                     ...       ...       ...       ...         ...  
control   F

In [21]:
df_count.reset_index().groupby('Condition').size()

Condition
100nM       978
control    1029
dtype: int64

In [22]:
df_count_cyto_nuclei

Nuclei                                                   \
PPI               AKT/Mtor Bim/Tom20 Cyclin D1/CDK4 Cyclin E/CDK2 EGFR/GRB2   
Condition FOV Id                                                              
100nM     FW1 36       0.0       3.0            0.0           1.0       2.0   
              37       2.0       2.0            0.0           2.0       4.0   
              41       2.0       0.0            1.0           0.0       2.0   
              47       3.0       4.0            6.0           3.0       4.0   
              53       1.0       4.0            6.0           5.0       3.0   
...                    ...       ...            ...           ...       ...   
control   FW2 599      0.0       0.0            0.0           0.0       0.0   
              602      0.0       0.0            0.0           0.0       0.0   
              605      0.0       0.0            0.0           0.0       0.0   
              608      0.0       0.0            0.0           0.0       0.0   
              609      0.0       0.0            0.0           0.0       0.0   

                                                                          ...  \
PPI               FoxO1/AKT Mcl-1/BAK NF-Kb/p-P90rsk SIRT1/P53 Sox2/Oct4  ...   
Condition FOV Id                                                          ...   
100nM     FW1 36        2.0       0.0            0.0       0.0       1.0  ...   
              37        6.0       1.0            3.0       0.0       1.0  ...   
              41        0.0       1.0            2.0       0.0       0.0  ...   
              47        6.0       3.0           10.0       0.0       3.0  ...   
              53        8.0       3.0            8.0       0.0       0.0  ...   
...                     ...       ...            ...       ...       ...  ...   
control   FW2 599       0.0       0.0            0.0       0.0       0.0  ...   
              602       0.0       0.0            0.0       0.0       0.0  ...   
              605       0.0       0.0            0.0       0.0       0.0  ...   
              608       0.0       0.0            0.0       0.0       0.0  ...   
              609       0.0       0.0            0.0       0.0       0.0  ...   

                           Cyto                                               \
PPI               Cyclin E/CDK2 EGFR/GRB2 FoxO1/AKT Mcl-1/BAK NF-Kb/p-P90rsk   
Condition FOV Id                                                               
100nM     FW1 36            0.0       6.0       6.0       4.0            5.0   
              37            1.0       6.0      -3.0       1.0            1.0   
              41            5.0       3.0       3.0       4.0            0.0   
              47            1.0       4.0      -2.0       1.0           -8.0   
              53            1.0       8.0       0.0      -1.0           -3.0   
...                         ...       ...       ...       ...            ...   
control   FW2 599           6.0      25.0       2.0       3.0            5.0   
              602           5.0      17.0      15.0       7.0            9.0   
              605           1.0       7.0       4.0       1.0            0.0   
              608          11.0      12.0       4.0       4.0            9.0   
              609           6.0      10.0       7.0       5.0            3.0   

                                                                       
PPI               SIRT1/P53 Sox2/Oct4 TEAD1/YAP TRAIL/DR5 p-ERK/c-MYC  
Condition FOV Id                                                       
100nM     FW1 36        0.0       0.0       1.0       7.0        11.0  
              37        0.0       2.0       0.0       0.0         8.0  
              41        0.0       0.0       0.0       3.0        40.0  
              47        0.0      -2.0       0.0      -4.0         4.0  
              53        0.0       6.0       1.0      -3.0        13.0  
...                     ...       ...       ...       ...         ...  
control  

In [23]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [24]:
# Run model on cell count
X = df_count_cell
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cell', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.8209
auc,0.90173
b_accuracy,0.82089
f1,0.82


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.77612
auc,0.86254
b_accuracy,0.77604
f1,0.78049


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.83541
auc,0.90188
b_accuracy,0.83365
f1,0.84862


wandb: 
wandb: Plotting MLP_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.79551
auc,0.88194
b_accuracy,0.79356
f1,0.81364


wandb: 
wandb: Plotting MLP_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\thu71\AppData\Local\anaconda3\envs\PLA2\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.81796
auc,0.88878
b_accuracy,0.81885
f1,0.81425
